# Server interaction

In [ ]:
import paramiko
from pathlib import Path

## Using SSH to execute remote commands

**Important note:** some servers have a limit on the number of successful or unsuccessful logins for a time period.  If that limit is exceeded, your IP address will be banned for some time.  Hence be careful never to make a large number of connections over a short period of time!

**Important note:** the key to use for authentication by Paramiko should be the first one in your SSH agent.  If necessary, delete other keys using `ssh-add -d <key>` for the other keys.  If necessary, they can be added again later.

Create a client, load the authentication keys, and connect to the system.

In [ ]:
client = paramiko.SSHClient()
client.load_system_host_keys()

In [ ]:
hostname = 'login.hpc.kuleuven.be'

In [ ]:
client.connect(hostname, username='vsc30140', )

Execute an `ls` command on the remote host.

In [ ]:
_, stdout, _ = client.exec_command('ls -l *.pbs')

Now the output of the remote `ls` command can be read from `stdout`.  Note that in this case we are ignoring both standard input and standard error.

In [ ]:
for line in stdout:
    print(line.strip())

### Dealing with errors

You can check the exit status of the command that was executed, the next one will not exit succesfully.  You can also access standard error.

In [ ]:
_, stdout, stderr = client.exec_command('ls -l this_file_certainly_does_not_exists.txt')

In [ ]:
stdout.channel.recv_exit_status()

In [ ]:
print(''.join(stderr).rstrip())

### Using standard input

Do an `wc` on a local file that will be standard input to the remote command, reading the result back in.

In [ ]:
stdin, stdout, _ = client.exec_command('wc')

Send the input to the remote `wc` by writing to `stdin`.  Once done, the channel for `stdin` should be shut down.

In [ ]:
with open('julia_omp.f90', 'r') as file:
    for line in file:
        print(line, file=stdin, end='')
stdin.channel.shutdown_write()

Now the output from the command can be read from `stdout`.

In [ ]:
print(''.join(stdout).rstrip())

Compare to the local result.

In [ ]:
!wc julia_omp.f90

In [ ]:
client.close()

## Using SFTP to transfer files

In [ ]:
client.connect(hostname, username='vsc30140')

Open an SFTP client to the remote host, and put a file on the system.

In [ ]:
sftp_client = client.open_sftp()

Define paths to the input file and output file.

In [ ]:
input_path = Path('julia_omp.f90')
output_path = Path('julia_omp_wc.txt')

In [ ]:
_ = sftp_client.put(str(input_path), str(input_path))

Check whether the input file is on the remote host.

In [ ]:
_, stdout, stderr = client.exec_command(f'ls -l {input_path}')
print(''.join(stdout).rstrip())
print(''.join(stderr).rstrip())

Compute the word count, redirecting the output to a file.

In [ ]:
_, _, stderr = client.exec_command(f'wc {input_path} > {output_path}')

Show standard error.

In [ ]:
print(''.join(stderr).rstrip())

Transfer the result back to the local host.

In [ ]:
sftp_client.get(str(output_path), str(output_path))

In [ ]:
!cat julia_omp_wc.txt

Remove the local output file.

In [ ]:
output_path.unlink()

Remove the remote input and output files.

In [ ]:
_, _, _ = client.exec_command(f'rm {input_path} {output_path}')

Close the SFTP client and the SSH client.

In [ ]:
sftp_client.close()
client.close()